## Build Environment

In [ ]:
# BUILD ENV
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install captionizer==1.0.1

!pip install --upgrade protobuf==3.20.0
!pip install tensorboard
%pip install gdown


## Dataset

In [1]:
dataset="person"


## Download Model
To rename to `model.ckpt`.


In [ ]:
# 1.5 Model 7Gb
# !gdown https://drive.google.com/uc?id=1LvIDeU4g3hjn1sxHiD7KSKA11UNafnLR


## Download Regularisation images
To put in `regularization_images/{dataset}`


In [ ]:
# Regularisation: creature
# !gdown https://drive.google.com/uc?id=1-XDTDCxN4HLYkcKPIY2G15M_fOISy4yU

# Regularisation: person
# !gdown https://drive.google.com/uc?id=1vVp9xHmUMXVxP7lNKw4QR2mpCq3C7Ts6

# Download Regularization Images (person_ddim is recommended)
# dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "blonde_woman"]
# !git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git
# !mkdir -p regularization_images/{dataset}
# !mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

# Upload your training images
Upload 10-20 images of someone to

```
/workspace/Dreambooth-Stable-Diffusion/training_images
```

WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body
*   5-12 close-up on face

The images should be:

- as close as possible to the kind of images you're trying to make

In [ ]:
#@markdown Add here the URLs to the images of the subject you are adding
urls = [
 "https://docteeboh.net/tmp/01.png",
 "https://docteeboh.net/tmp/02.png",
 "https://docteeboh.net/tmp/03.png",
 "https://docteeboh.net/tmp/04.png",
 "https://docteeboh.net/tmp/05.png",
 "https://docteeboh.net/tmp/06.png",
 "https://docteeboh.net/tmp/07.png",
 "https://docteeboh.net/tmp/08.png",
 "https://docteeboh.net/tmp/09.png",
 "https://docteeboh.net/tmp/10.png",
 "https://docteeboh.net/tmp/11.png",
 "https://docteeboh.net/tmp/12.png",
 "https://docteeboh.net/tmp/13.png",
 "https://docteeboh.net/tmp/14.png",
 "https://docteeboh.net/tmp/15.png",
 "https://docteeboh.net/tmp/16.png",
 "https://docteeboh.net/tmp/17.png",
 "https://docteeboh.net/tmp/18.png",
 "https://docteeboh.net/tmp/19.png",
 "https://docteeboh.net/tmp/20.png",
 "https://docteeboh.net/tmp/21.png",
 "https://docteeboh.net/tmp/22.png",
 "https://docteeboh.net/tmp/23.png",
 "https://docteeboh.net/tmp/24.png",
 "https://docteeboh.net/tmp/25.png",
 "https://docteeboh.net/tmp/26.png",
 "https://docteeboh.net/tmp/27.png",
 "https://docteeboh.net/tmp/28.png",
 "https://docteeboh.net/tmp/29.png",
 "https://docteeboh.net/tmp/30.png",
 "https://docteeboh.net/tmp/31.png",
 "https://docteeboh.net/tmp/32.png",
 "https://docteeboh.net/tmp/33.png",
 "https://docteeboh.net/tmp/34.png",
 "https://docteeboh.net/tmp/35.png",
 "https://docteeboh.net/tmp/36.png",
 "https://docteeboh.net/tmp/37.png",
 "https://docteeboh.net/tmp/38.png",
 "https://docteeboh.net/tmp/39.png",
 "https://docteeboh.net/tmp/40.png",
 "https://docteeboh.net/tmp/41.png",
 "https://docteeboh.net/tmp/42.png",
 "https://docteeboh.net/tmp/43.png",
 "https://docteeboh.net/tmp/44.png",
 "https://docteeboh.net/tmp/45.png",
 "https://docteeboh.net/tmp/46.png",
 "https://docteeboh.net/tmp/47.png",
 "https://docteeboh.net/tmp/48.png",
 "https://docteeboh.net/tmp/49.png",
 "https://docteeboh.net/tmp/50.png",
 "https://docteeboh.net/tmp/51.png",
 "https://docteeboh.net/tmp/52.png",
 "https://docteeboh.net/tmp/53.png",
 "https://docteeboh.net/tmp/54.png",
 "https://docteeboh.net/tmp/55.png",
 "https://docteeboh.net/tmp/56.png",
 "https://docteeboh.net/tmp/57.png",
 "https://docteeboh.net/tmp/58.png",
 "https://docteeboh.net/tmp/59.png",
 "https://docteeboh.net/tmp/60.png",
 "https://docteeboh.net/tmp/61.png",
 # You can add additional images here -- about 20-30 images in different
]

In [ ]:
#@title Download and check the images you have just added
import os
import requests
from io import BytesIO
from PIL import Image


def image_grid(imgs, rows, cols):
 assert len(imgs) == rows*cols

 w, h = imgs[0].size
 grid = Image.new('RGB', size=(cols*w, rows*h))
 grid_w, grid_h = grid.size

 for i, img in enumerate(imgs):
  grid.paste(img, box=(i%cols*w, i//cols*h))
 return grid

def download_image(url):
 try:
  response = requests.get(url)
 except:
  return None
 return Image.open(BytesIO(response.content)).convert("RGB")

images = list(filter(None,[download_image(url) for url in urls]))
save_path = "./training_images"
if not os.path.exists(save_path):
 os.mkdir(save_path)
[image.save(f"{save_path}/{i}.png", format="png") for i, image in enumerate(images)]
image_grid(images, 1, len(images))

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [ ]:
# Training

# This isn't used for training, just to help you remember what your trained into the model.
project_name = "project_name"

# MAX STEPS
# How many steps do you want to train for?
max_training_steps = 2000

# Match class_word to the category of the regularization images you chose above.
class_word = "person" # typical uses are "man", "person", "woman"

# This is the unique token you are incorporating into the stable diffusion model.
token = "firstNameLastName"


reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "{reg_data_root}" \
 -n "{project_name}" \
 --gpus 0, \
 --data_root "/workspace/Dreambooth-Stable-Diffusion/training_images" \
 --max_training_steps {max_training_steps} \
 --class_word "{class_word}" \
 --token "{token}" \
 --no-test

## Copy and name the checkpoint file

In [ ]:
# Copy the checkpoint into our `trained_models` folder

directory_paths = !ls -d logs/*
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_images)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + token + "_token_" + class_word + "_class_word.ckpt"

file_name = file_name.replace(" ", "_")

!mkdir -p trained_models
!mv "{last_checkpoint_file}" "trained_models/{file_name}"

print("Download your trained model file from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")

# Optional - Upload to google drive
* run the following commands in a new `terminal` in the `Dreambooth-Stable-Diffusion` directory
* `chmod +x ./gdrive`
* `./gdrive about`
* `paste your token here after navigating to the link`
* `./gdrive upload trained_models/{file_name.ckpt}`

# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`

## Generate Images With Your Trained Model!

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter 4 \
 --scale 7.0 \
 --ddim_steps 50 \
 --ckpt "/workspace/Dreambooth-Stable-Diffusion/trained_models/{file_name}" \
 --prompt "joepenna person as a masterpiece portrait painting by John Singer Sargent in the style of Rembrandt"